In [14]:
import sklearn
from sklearn.datasets import load_boston
from sklearn.linear_model import Lasso, ElasticNet, Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [8]:
boston = load_boston()
boston_df = pd.DataFrame(boston.data, columns=boston.feature_names)
boston_df['target'] = boston.target

data = boston_df.drop('target', axis = 1, inplace=False)
target = boston_df['target']

In [51]:
def regression(model_name, params = None, data=None, target=None, vervose=True):
  coeff_df = pd.DataFrame()
  if vervose : print('######', model_name, '######')
  for param in params:
    if model_name == 'Ridge' : model=Ridge(alpha=param)
    elif model_name == 'Lasso' : model=Lasso(alpha=param)
    elif model_name == 'ElasticNet' : model=ElasticNet(alpha=param, l1_ratio=0.7)
    n_mse = cross_val_score(model, data, target, scoring='neg_mean_squared_error', cv=5)
    avg_rmse = np.mean(np.sqrt(-1*n_mse))
    print('alpha={0} : RMSE = {1:.3f}'.format(param, avg_rmse))

    model.fit(data, target)
    coeff = pd.Series(data=model.coef_, index=data.columns)
    colname = 'alpha:'+str(param)
    coeff_df[colname]=coeff
  return coeff_df

In [56]:
lasso_alphas = [0.07, 0.1, 0.5, 1, 3]
lasso_coeff_df = regression('Lasso', params=lasso_alphas, data = data, target = target)

sort_colname = 'alpha:'+str(lasso_alphas[0])
lasso_coeff_df.sort_values(by=sort_colname, ascending=False)

###### Lasso ######
alpha=0.07 : RMSE = 5.612
alpha=0.1 : RMSE = 5.615
alpha=0.5 : RMSE = 5.669
alpha=1 : RMSE = 5.776
alpha=3 : RMSE = 6.189


,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
RM,3.789725,3.703202,2.498212,0.949811,0.000000
CHAS,1.434343,0.955190,0.000000,0.000000,0.000000
RAD,0.270936,0.274707,0.277451,0.264206,0.061864
ZN,0.049059,0.049211,0.049544,0.049165,0.037231
B,0.010248,0.010249,0.009469,0.008247,0.006510
NOX,-0.000000,-0.000000,-0.000000,-0.000000,0.000000
AGE,-0.011706,-0.010037,0.003604,0.020910,0.042495
TAX,-0.014290,-0.014570,-0.015442,-0.015212,-0.008602
INDUS,-0.042120,-0.036619,-0.005253,-0.000000,-0.000000
CRIM,-0.098193,-0.097894,-0.083289,-0.063437,-0.000000


In [57]:
elastic_alphas = [0.07, 0.1, 0.5, 1, 3]
elastic_coeff_df = regression('ElasticNet', params=elastic_alphas, data=data, target=target)

sort_colname_2 = 'alpha:'+str(elastic_alphas[0])
elastic_coeff_df.sort_values(by=sort_colname_2, ascending=False)

###### ElasticNet ######
alpha=0.07 : RMSE = 5.542
alpha=0.1 : RMSE = 5.526
alpha=0.5 : RMSE = 5.467
alpha=1 : RMSE = 5.597
alpha=3 : RMSE = 6.068


,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
RM,3.574162,3.414154,1.918419,0.938789,0.000000
CHAS,1.330724,0.979706,0.000000,0.000000,0.000000
RAD,0.278880,0.283443,0.300761,0.289299,0.146846
ZN,0.050107,0.050617,0.052878,0.052136,0.038268
B,0.010122,0.010067,0.009114,0.008320,0.007020
AGE,-0.010116,-0.008276,0.007760,0.020348,0.043446
TAX,-0.014522,-0.014814,-0.016046,-0.016218,-0.011417
INDUS,-0.044855,-0.042719,-0.023252,-0.000000,-0.000000
CRIM,-0.099468,-0.099213,-0.089070,-0.073577,-0.019058
NOX,-0.175072,-0.000000,-0.000000,-0.000000,-0.000000
